<h1 style="text-align: center;">Analyis of Point Production of NBA Players</h1>

# Introduction

Using this dataset of NBA player stats we want to perform a regression analysis of the selected variables to help predict the on-court performance of a player (their average points scored). To do so, all non-preformance related stats, including, age,  height, weight, and draft pick will be used. The season variable will also allow player preformances to be measured throughout time and compare NBA starts from all eras and styles of the game. All of these factors were selected because of their likelihood to correlate with stronger on-court performance.

Our question: How do our selected factors help to predict the individual performance of NBA players? 

To do this, we will use [this dataset](https://www.kaggle.com/datasets/justinas/nba-players-data) from Kaggle. The data was acquired through the NBA API with missing information supplied by scraping from another NBA source (for this reason, the dataset was taken directly from Kaggle and not the original source). 

# Methods and Results

## Load Libraries

In [2]:
library(tidyverse)
library(tidymodels)
library(repr)
options(repr.matrix.max.rows = 6)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(tidymodels): there is no package called ‘tidymodels’


## Dataset Load & Preperation

First we pull our dataset from the internet.

The dataset contains 22 variables, the majority of which we need to discard as being impractical to use, having little impact on player performance (player name) or being directly related to a performance variable. We will conduct analysis on the remaining selected variables to try and predict the average points a player will get per season using regression analysis. By first training a model on the training data for each respective datapoint category we will then be able to analyze the knn regression and decide which of the columns is most effective at indicating points production.

In [ ]:
url <-"https://raw.githubusercontent.com/mdean808/dsci-100-group-project/b11c50b091b2c4a554a2b7ff8f9e568e081b0f3c/all_seasons.csv"


temp <- tempfile()


download.file(url, temp)
# read the dataset from temp file
player_data <- read_csv(temp)
head(player_data)

For our project, we will focus only on the specific parameters to judge the overall performance of an NBA player. 

Note that for `draft_round` players labeled as `Undrafted` were removed. Additionally, players drafted in or before 2000 are excluded, to keep the data used recent.

In [ ]:
nba_players <- player_data |>
    filter(draft_year > 2000) |>
    select(age, player_height, player_weight, net_rating, draft_number, draft_round, pts) |>
    # remove undrafted players and convert draft_number to dbl
    filter(draft_round != "Undrafted") |>
    mutate(draft_round = as.numeric(draft_round)) |>
    mutate(draft_number = as.numeric(draft_number))
                
nba_players

Now we will split the data into training and testing. We will use the training data to build our regression model and our testing data to test the validity of our model. 

In [ ]:
set.seed("1234")
nba_players_split <- initial_split(nba_players, prop = 0.75, strata = pts)
nba_training <- training(nba_players_split)
nba_testing <- testing(nba_players_split)

nba_training

# Dataset Visualization

Now we will look at the correlation of multiple parameters compared with points. This will allow us to predict which parameters strongly influence overall points scored before we build our regression model. 

In [ ]:
options(repr.plot.width = 10)

nba_plot_age <- ggplot(nba_training, aes(x = age, y = pts)) +
geom_point() +
labs(x = "Age of player (yrs)", y = "Avg. points/game") +
ggtitle("Age of players vs. Avg. Points") +
theme(text = element_text(size = 12))

nba_plot_height <- ggplot(nba_training, aes(x = player_height, y = pts)) +
geom_point() +
labs(x = "Player height (cm)", y = "Avg. points/game") +
ggtitle("Player height vs. Avg. Points") +
theme(text = element_text(size = 12))

nba_plot_weight <- ggplot(nba_training, aes(x = player_weight, y = pts)) +
geom_point() +
labs(x = "Player weight (kg)", y = "Avg. points/game") +
ggtitle("Player weight vs. Avg. Points") +
theme(text = element_text(size = 12))


nba_plot_number <- nba_training |>
mutate(draft_number = as.numeric(draft_number)) |>
ggplot(aes(x = draft_number, y = pts)) +
geom_point() +
labs(x = "Draft Position", y = "Avg. points/game") +
ggtitle("Draft Position vs. Avg. Points") +
theme(text = element_text(size = 12))


nba_plot_nr <- ggplot(nba_training, aes(x = net_rating, y = pts)) +
geom_point() +
labs(x = "Net rating", y = "Avg. points/game") +
ggtitle("Net rating vs. Avg. Points") +
theme(text = element_text(size = 12))


nba_plot_dr <- ggplot(nba_training, aes(x = draft_round, y = pts)) +
geom_point() +
labs(x = "Draft Round", y = "Avg. points/game") +
ggtitle("Draft Round vs. Avg. Points") +
theme(text = element_text(size = 12))

nba_plot_age
nba_plot_height
nba_plot_weight
nba_plot_number
nba_plot_nr
nba_plot_dr

As shown in the graphs, younger players and earlier draft position have positive correlation with average points. We expect these two parameters to be good predictors for performance. Basic information about the dataset is presented in the following table, consisting of the means for age, weight, height, draft number, as well as how many data points are contained in the set. 

In [ ]:
summarise_age <- summarise(nba_training, mean_age = mean(age))
summarise_height <- summarise(nba_training, mean_height = mean(player_height))
summarise_weight <- summarise(nba_training, mean_weight = mean(player_weight))
summarise_nr <- summarise(nba_training, mean_nr = mean(net_rating))
summarise_nrow <- summarise(nba_training, total_rows = nrow(nba_training))

summarise_draft_number <- nba_training |>
summarise(mean_number = mean(as.numeric(draft_number), na.rm = TRUE))

summarise_draft_round <- nba_training |>
summarise(mean_round = mean(as.numeric(draft_round), na.rm = TRUE))



combined_summary <- bind_cols(summarise_age, summarise_weight, summarise_height, summarise_draft_number, summarise_nr, summarise_nrow, 
                             summarise_draft_round)


combined_summary



## Data Analysis

To predict the player performance variable, we will be using k-nearest neighbors regression, as the relationships between the predictor variables and our performance variable are not all linear, and the performance variable is numeric.

First, we prepare the recipe by using the (scaled) age, player height, player weight, draft number, and draft round to predict the player's average points.

In [ ]:
nba_recipe <- recipe(pts ~ age + player_height + player_weight + net_rating + draft_number + draft_round, data = nba_training) |>
step_scale(all_predictors()) |>
step_center(all_predictors())

nba_recipe

Then, we prepare the model. First, we will using tuning to determine the best number of neighbours to use, employing 5-fold cross-validation.

In [ ]:
nba_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
set_engine("kknn") |>
set_mode("regression")

nba_vfold <- vfold_cv(nba_training, v = 5, strata = pts)

nba_workflow <- workflow() |>
add_recipe(nba_recipe) |>
add_model(nba_spec)

nba_workflow


Finally, we put it all together, using between 1 to 150 neighbours in increments of 5.

In [ ]:
k_vals <- tibble(neighbors = seq(from = 1, to = 150, by = 5))

nba_results <- nba_workflow |>
  tune_grid(resamples = nba_vfold, grid = k_vals) |>
  collect_metrics() |>
  filter(.metric == "rmse")

nba_results

nba_smallest <- nba_results |>
  filter(mean == min(mean))
nba_smallest


Next, we prepare the final model, using the optimal amount of neighbors.

In [ ]:
neighbors_val <- nba_smallest |>
pull(neighbors)

nba_tuned_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = neighbors_val) |>
  set_engine("kknn") |>
  set_mode("regression")

nba_fit <- workflow() |>
  add_recipe(nba_recipe) |>
  add_model(nba_tuned_spec) |>
  fit(data = nba_training)

nba_predict <- nba_fit |>
  predict(nba_testing) |>
  bind_cols(nba_testing)



nba_rmse <- nba_predict |>
  metrics(truth = pts, estimate = .pred) |>
  filter(.metric == 'rmse')

nba_predict

nba_rmse

#head(nba_players |> arrange(pts, .desc = TRUE))

Due to the multiple variables used in prediction, the visualization for this prediction is difficult to produce. Instead, each variable used in the analysis is presented separately.

In [ ]:
nba_predict_plot_age <- ggplot(nba_training, aes(x = age, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = age, y = .pred), color = 'blue') +
labs(x = "Age (yrs)", y = "Points") +
ggtitle("Age vs. Points Scored with Estimated Values")


nba_predict_plot_age

!!! Comments here

In [ ]:
nba_predict_plot_height <- ggplot(nba_training, aes(x = player_height, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = player_height, y = .pred), color = 'blue') +
labs(x = "Height (cm)", y = "Points") +
ggtitle("Height (cm) vs. Points Scored with Estimated Values")

nba_predict_plot_height

In [ ]:
nba_predict_plot_weight <- ggplot(nba_training, aes(x = player_weight, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = player_weight, y = .pred), color = 'blue') +
labs(x = "Weight (kg)", y = "Points") +
ggtitle("Weight (kg) vs. Points Scored with Estimated Values")

nba_predict_plot_weight

In [ ]:
options(repr.plot.width = 18)

nba_predict_plot_nr <- ggplot(nba_training, aes(x = net_rating, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = net_rating, y = .pred), color = 'blue') +
labs(x = "Net Rating", y = "Points") +
ggtitle("Net Rating vs. Points Scored with Estimated Values")

nba_predict_plot_nr

In [ ]:
options(repr.plot.width = 10)
nba_predict_plot_dn <- ggplot(nba_training, aes(x = draft_number, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = draft_number, y = .pred), color = 'blue') +
labs(x = "Draft Number", y = "Points") +
ggtitle("Draft Number vs. Points Scored with Estimated Values")

nba_predict_plot_dn

In [ ]:

nba_predict_plot_dr <- ggplot(nba_training, aes(x = draft_round, y = pts)) +
geom_point() +
geom_line(data = nba_predict, mapping = aes(x = draft_round, y = .pred), color = 'blue') +
labs(x = "Draft Round", y = "Points") +
ggtitle("Draft Round vs. Points Scored with Estimated Values")
nba_predict_plot_dr

# delete this

# Discussion

* summarize what you found
* discuss whether this is what you expected to find?
* discuss what impact could such findings have?
* discuss what future questions could this lead to?

# References
At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
Make sure to cite the source of your data as well.